In [66]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from oauth2client.service_account import ServiceAccountCredentials
import time
import pyperclip
import random
import string
import pandas as pd
import gspread
import os

In [32]:
class GoogleSheetHandler:
    def __init__(self, credentials_file='credentials.json'):
        self.scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
        self.credentials_file = credentials_file
        self.credentials = ServiceAccountCredentials.from_json_keyfile_name(self.credentials_file, self.scope)
        self.gc = gspread.authorize(self.credentials)

    def get_sheet_data(self, spreadsheet_name, worksheet_index=0):
        spreadsheet = self.gc.open(spreadsheet_name)
        worksheet = spreadsheet.get_worksheet(worksheet_index)
        data = worksheet.get_all_values()
        df = pd.DataFrame(data[1:], columns=data[0])
        return df

    def overwrite_sheet_with_data(self, dataframe, spreadsheet_name, worksheet_index=0):
        spreadsheet = self.gc.open(spreadsheet_name)
        worksheet = spreadsheet.get_worksheet(worksheet_index)
        data = dataframe.values.tolist()
        worksheet.clear()
        # worksheet.resize(len(data), len(data[0]))
        worksheet.append_row(dataframe.columns.tolist())
        worksheet.append_rows(data)

In [90]:
handler = GoogleSheetHandler()
df = handler.get_sheet_data('TiktokBot')
df["Status"].replace("", "Remaining", inplace=True)
handler.overwrite_sheet_with_data(df, "TiktokBot")
df

,Topic,Status
0,School,Remaining
1,Greek Kings,Remaining
2,Dark Web,Remaining
3,Gold,Remaining
4,Horse,Remaining
...,...,...
99,Deosai Plains,Remaining
100,Deserts,Remaining
101,Mount Everest,Remaining
102,Maths,Remaining


In [34]:
def generate_password(length=10):
    characters = string.ascii_letters + string.digits + string.punctuation
    return ''.join(random.choice(characters) for _ in range(length))

In [35]:
co = uc.ChromeOptions()

co.add_argument("--start-maximized")
co.add_argument('--disable-gpu')
co.add_argument('--disable-infobars')
# co.add_argument('--no-sandbox')
co.add_argument("--disable-popup-blocking")
co.add_argument(r"--user-data-dir=C:\\Users\\Dell\\AppData\\Local\\Google\\Chrome\\User Data") #e.g. C:\Users\You\AppData\Local\Google\Chrome\User Data
co.add_argument(r'--profile-directory=Profile 10') #e.g. Profile 3

driver = uc.Chrome(options=co, use_subprocess=True)
actions = ActionChains(driver)
# URLs to visit
driver.get("https://chat.openai.com/c/202a469b-f42f-4bea-a736-ae4bfc7ebed3")
driver.implicitly_wait(10)
driver.execute_script(f'''window.open("https://internxt.com/temporary-email");''')
driver.switch_to.window(driver.window_handles[1])
driver.implicitly_wait(10)
driver.execute_script(f'''window.open("https://www.capcut.com/my-cloud/7341722566995918853");''')
driver.switch_to.window(driver.window_handles[2])
driver.implicitly_wait(10)
driver.execute_script(f'''window.open("https://app.simplified.com/signup");''')
driver.switch_to.window(driver.window_handles[3])
driver.implicitly_wait(10)

In [36]:
driver.switch_to.window(driver.window_handles[1])
time.sleep(2)
driver.find_element(By.XPATH, "//p[contains(text(), 'Delete')]/..").click()
time.sleep(1.2)
driver.find_element(By.XPATH, "//p[contains(text(), 'Copy')]/..").click()

In [37]:
driver.switch_to.window(driver.window_handles[3])
email = pyperclip.paste()

time.sleep(0.5)
driver.find_element(By.XPATH, "//input[@type='email']").send_keys(email)
password = generate_password()
driver.find_element(By.XPATH, "//input[@type='password']").send_keys(password)
driver.find_element(By.XPATH, "(//input[@type='password'])[2]").send_keys(password)

driver.find_element(By.XPATH, "//button[@type='submit']").click()

In [40]:
driver.switch_to.window(driver.window_handles[1])
VerificationCode = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//p[contains(text(),'Email Verification passcode')]"))).text.split("Your Email Verification passcode is ")[1].split(" Thank you")[0]

In [41]:
driver.switch_to.window(driver.window_handles[3])

driver.find_element(By.XPATH, "//input[@placeholder='Enter code here']").send_keys(VerificationCode)
driver.find_element(By.XPATH, "//button[@type='submit']").click()

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@placeholder='Hint: Name of your company']"))).send_keys(password)
driver.find_element(By.XPATH, "//button[@type='submit']").click()
time.sleep(1)

In [42]:
driver.get("https://app.simplified.com/activate/video")

driver.get("https://app.simplified.com/video/generate-ai/script-to-video")

In [80]:
topic = "Dark Web"

In [83]:
driver.switch_to.window(driver.window_handles[0])

prompts = [f'Write a 55 second viral educational showcase Tik Tok script about Do you know about "{topic}" Make sure the script is engaging with a good, quick hook. Keep the script relatively serious and educational. The script should ONLY include the voice, NOT the editing direction or stage direction! Start with a " and end with a ". Have paragraph breaks that stay within the". Do not do a countdown. Do not include an outro or summary. DO NOT USE EMOJIS! The script should portray the info in interactive way. Describe and make it educational. Start with "Do you know!"', "give me short caption for this too with suitable hashtags for tiktok with #DidYouKnow included"]

driver.find_element(By.XPATH, "//textarea[@placeholder='Message ChatGPT…']").send_keys(prompts[0])
driver.find_element(By.XPATH, "//button[@data-testid='send-button']").click()

time.sleep(1)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//button[@data-testid='send-button']")))
driver.find_elements(By.XPATH, "//*[@class='flex items-center gap-1.5 rounded-md p-1 text-xs text-token-text-tertiary hover:text-token-text-primary md:invisible md:group-hover:visible md:group-[.final-completion]:visible']")[-1].click()
script = pyperclip.paste()
print("Script: ", script)

driver.find_element(By.XPATH, "//textarea[@placeholder='Message ChatGPT…']").send_keys(prompts[1])
driver.find_element(By.XPATH, "//button[@data-testid='send-button']").click()

time.sleep(1)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//button[@data-testid='send-button']")))
driver.find_elements(By.XPATH, "//*[@class='flex items-center gap-1.5 rounded-md p-1 text-xs text-token-text-tertiary hover:text-token-text-primary md:invisible md:group-hover:visible md:group-[.final-completion]:visible']")[-1].click()
title = pyperclip.paste()[1:-1]
print("Title: ", title)


driver.switch_to.window(driver.window_handles[3])
driver.get("https://app.simplified.com/video/generate-ai/script-to-video")
driver.implicitly_wait(10)
time.sleep(5)
driver.find_element(By.XPATH, "//*[@id='popover-trigger-17']").click()
WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//button[contains(text(),'AI Speakers')]"))).click()
WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Matthew')]/../../../.."))).click()
time.sleep(0.5)

driver.find_element(By.XPATH, "//textarea[@name='title']").click()
pyperclip.copy(title)
actions.key_down(Keys.CONTROL).send_keys("v").key_up(Keys.CONTROL).perform()

driver.find_element(By.XPATH, "//textarea[@name='description']").click()
pyperclip.copy(script)
actions.key_down(Keys.CONTROL).send_keys("v").key_up(Keys.CONTROL).perform()

WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Generate')]"))).click()

Script:  "Do you know! Let's dive into the shadowy depths of the Dark Web. It's like the internet's secret underworld, accessible only through specialized software and encryption. Unlike the websites we use every day, the Dark Web operates in the shadows, away from the prying eyes of search engines and authorities.

Here, anonymity reigns supreme, making it a haven for all sorts of illicit activities. From illegal drug markets to cybercrime forums, the Dark Web hosts a myriad of shady dealings. But it's not all black market transactions; the Dark Web also serves as a refuge for whistleblowers and activists who need to communicate securely without fear of reprisal.

However, traversing the Dark Web comes with significant risks. It's a digital wild west, where scams, malware, and other dangers lurk in every corner. Law enforcement agencies worldwide are constantly monitoring the Dark Web to combat criminal activity and protect internet users.

Understanding the Dark Web is essential in t

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00938C93+51395]
	(No symbol) [0x008A5EF1]
	(No symbol) [0x0075E13A]
	(No symbol) [0x007962BB]
	(No symbol) [0x007963EB]
	(No symbol) [0x007CC162]
	(No symbol) [0x007B3ED4]
	(No symbol) [0x007CA570]
	(No symbol) [0x007B3C26]
	(No symbol) [0x0078C629]
	(No symbol) [0x0078D40D]
	GetHandleVerifier [0x00CB6453+3711107]
	GetHandleVerifier [0x00CF583A+3970154]
	GetHandleVerifier [0x00CF0B28+3950424]
	GetHandleVerifier [0x009E9C99+776393]
	(No symbol) [0x008B16C4]
	(No symbol) [0x008AC5A8]
	(No symbol) [0x008AC759]
	(No symbol) [0x0089DD80]
	BaseThreadInitThunk [0x7576FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77307C5E+286]
	RtlGetAppContainerNamedObjectPath [0x77307C2E+238]


In [ ]:
WebDriverWait(driver, 600).until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(),'Export')]/.."))).click()
WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Download')]"))).click()

In [ ]:
def find_first_mp4(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".mp4"):
                return os.path.join(root, file)
    return None

# Function to rename a file
def rename_file(old_path, new_name):
    os.rename(old_path, new_name)

In [ ]:
while True:
    if find_first_mp4("media") is not None:break

In [ ]:
rename_file(find_first_mp4("media"),f"media/{title}.mp4")
path = os.path.join(os.getcwd(), find_first_mp4("media"))

In [ ]:
driver.switch_to.window(driver.window_handles[2])

In [ ]:
driver.find_element(By.XPATH, "//input[@type='file']").send_keys(path)

In [14]:
# %%
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from oauth2client.service_account import ServiceAccountCredentials
import time
import pyperclip
import random
import string
import pandas as pd
import gspread
import os

# %%
class GoogleSheetHandler:
    def __init__(self, credentials_file='credentials.json'):
        self.scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
        self.credentials_file = credentials_file
        self.credentials = ServiceAccountCredentials.from_json_keyfile_name(self.credentials_file, self.scope)
        self.gc = gspread.authorize(self.credentials)

    def get_sheet_data(self, spreadsheet_name, worksheet_index=0):
        spreadsheet = self.gc.open(spreadsheet_name)
        worksheet = spreadsheet.get_worksheet(worksheet_index)
        data = worksheet.get_all_values()
        df = pd.DataFrame(data[1:], columns=data[0])
        return df

    def overwrite_sheet_with_data(self, dataframe, spreadsheet_name, worksheet_index=0):
        spreadsheet = self.gc.open(spreadsheet_name)
        worksheet = spreadsheet.get_worksheet(worksheet_index)
        data = dataframe.values.tolist()
        worksheet.clear()
        # worksheet.resize(len(data), len(data[0]))
        worksheet.append_row(dataframe.columns.tolist())
        worksheet.append_rows(data)

# %%
handler = GoogleSheetHandler()
df = handler.get_sheet_data('TiktokBot')
df["Status"].replace("", "Remaining", inplace=True)
handler.overwrite_sheet_with_data(df, "TiktokBot")
df

# %%
def generate_password(length=10):
    characters = string.ascii_letters + string.digits + string.punctuation
    return ''.join(random.choice(characters) for _ in range(length))

# %%
co = uc.ChromeOptions()

co.add_argument("--start-maximized")
co.add_argument('--disable-gpu')
co.add_argument('--disable-infobars')
co.add_argument('--no-sandbox')
# co.add_argument('--headless')
co.add_argument("--disable-popup-blocking")
co.add_argument(r"--user-data-dir=C:\\Users\\Dell\\AppData\\Local\\Google\\Chrome\\User Data") #e.g. C:\Users\You\AppData\Local\Google\Chrome\User Data
co.add_argument(r'--profile-directory=Profile 10') #e.g. Profile 3

driver = uc.Chrome(options=co, use_subprocess=True)
actions = ActionChains(driver)
driver.maximize_window()
# URLs to visit
driver.get("https://chat.openai.com/c/202a469b-f42f-4bea-a736-ae4bfc7ebed3")
driver.implicitly_wait(10)
driver.execute_script(f'''window.open("https://internxt.com/temporary-email");''')
driver.switch_to.window(driver.window_handles[1])
driver.implicitly_wait(10)
driver.execute_script(f'''window.open("https://www.capcut.com/my-cloud/7341722566995918853");''')
time.sleep(5)
driver.switch_to.window(driver.window_handles[2])
driver.implicitly_wait(10)
driver.execute_script(f'''window.open("https://app.simplified.com/signup");''')
driver.switch_to.window(driver.window_handles[3])
driver.implicitly_wait(10)

# %%
driver.switch_to.window(driver.window_handles[1])
time.sleep(2)
driver.find_element(By.XPATH, "//p[contains(text(), 'Delete')]/..").click()
time.sleep(1.2)
driver.find_element(By.XPATH, "//p[contains(text(), 'Copy')]/..").click()

# %%
driver.switch_to.window(driver.window_handles[3])
email = pyperclip.paste()

time.sleep(0.5)
driver.find_element(By.XPATH, "//input[@type='email']").send_keys(email)
password = generate_password()
driver.find_element(By.XPATH, "//input[@type='password']").send_keys(password)
driver.find_element(By.XPATH, "(//input[@type='password'])[2]").send_keys(password)

driver.find_element(By.XPATH, "//button[@type='submit']").click()

# %%
driver.switch_to.window(driver.window_handles[1])
VerificationCode = WebDriverWait(driver, 300).until(EC.presence_of_element_located((By.XPATH, "//p[contains(text(),'Email Verification passcode')]"))).text.split("Your Email Verification passcode is ")[1].split(" Thank you")[0]

# %%
driver.switch_to.window(driver.window_handles[3])

driver.find_element(By.XPATH, "//input[@placeholder='Enter code here']").send_keys(VerificationCode)
driver.find_element(By.XPATH, "//button[@type='submit']").click()

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@placeholder='Hint: Name of your company']"))).send_keys(password)
driver.find_element(By.XPATH, "//button[@type='submit']").click()
time.sleep(1)

# %%
driver.get("https://app.simplified.com/activate/video")

driver.get("https://app.simplified.com/video/generate-ai/script-to-video")

# %%
DoneCnt = 0
for cnt, topic in  enumerate(df["Topic"]):
    if df["Status"].values.tolist()[cnt] == "Done":continue
    driver.switch_to.window(driver.window_handles[0])

    prompts = [f'Write a 55 second viral educational showcase Tik Tok script about Do you know about "{topic}" Make sure the script is engaging with a good, quick hook. Keep the script relatively serious and educational. The script should ONLY include the voice, NOT the editing direction or stage direction! Start with a " and end with a ". Have paragraph breaks that stay within the". Do not do a countdown. Do not include an outro or summary. DO NOT USE EMOJIS! The script should portray the info in interactive way. Describe and make it educational. Start with "Do you know!"', "give me short caption for this too with suitable hashtags for tiktok with #DidYouKnow included"]

    driver.find_element(By.XPATH, "//textarea[@placeholder='Message ChatGPT…']").send_keys(prompts[0])
    driver.find_element(By.XPATH, "//button[@data-testid='send-button']").click()

    time.sleep(1)
    WebDriverWait(driver, 50).until(EC.presence_of_element_located((By.XPATH, "//button[@data-testid='send-button']")))
    driver.find_elements(By.XPATH, "//*[@class='flex items-center gap-1.5 rounded-md p-1 text-xs text-token-text-tertiary hover:text-token-text-primary md:invisible md:group-hover:visible md:group-[.final-completion]:visible']")[-1].click()
    script = pyperclip.paste()
    print("Script: ", script)

    driver.find_element(By.XPATH, "//textarea[@placeholder='Message ChatGPT…']").send_keys(prompts[1])
    driver.find_element(By.XPATH, "//button[@data-testid='send-button']").click()

    time.sleep(1)
    WebDriverWait(driver, 50).until(EC.presence_of_element_located((By.XPATH, "//button[@data-testid='send-button']")))
    driver.find_elements(By.XPATH, "//*[@class='flex items-center gap-1.5 rounded-md p-1 text-xs text-token-text-tertiary hover:text-token-text-primary md:invisible md:group-hover:visible md:group-[.final-completion]:visible']")[-1].click()
    title = pyperclip.paste()[1:-1]
    print("Title: ", title)

    driver.switch_to.window(driver.window_handles[3])
    driver.get("https://app.simplified.com/video/generate-ai/script-to-video")
    driver.implicitly_wait(10)
    time.sleep(5)
    driver.find_element(By.XPATH, "//*[@id='popover-trigger-17']").click()
    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//button[contains(text(),'AI Speakers')]"))).click()
    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Matthew')]/../../../.."))).click()
    time.sleep(0.5)

    driver.find_element(By.XPATH, "//textarea[@name='title']").click()
    pyperclip.copy(title)
    actions.key_down(Keys.CONTROL).send_keys("v").key_up(Keys.CONTROL).perform()

    driver.find_element(By.XPATH, "//textarea[@name='description']").click()
    pyperclip.copy(script)
    actions.key_down(Keys.CONTROL).send_keys("v").key_up(Keys.CONTROL).perform()

    WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Generate')]"))).click()

    # %%
    WebDriverWait(driver, 600).until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(),'Export')]/.."))).click()
    WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Download')]"))).click()

    # %%
    def find_first_mp4(folder_path):
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if file.endswith(".mp4"):
                    return os.path.join(root, file)
        return None

    # Function to rename a file
    def rename_file(old_path, new_name):
        os.rename(old_path, new_name)

    # %%
    while True:
        if find_first_mp4("media") is not None:break

    # %%
    rename_file(find_first_mp4("media"),f"media/{title}.mp4")
    path = os.path.join(os.getcwd(), find_first_mp4("media"))

    # %%
    driver.switch_to.window(driver.window_handles[2])

    # %%
    driver.find_element(By.XPATH, "//input[@type='file']").send_keys(path)

    time.sleep(10)
    os.remove(path)

    # %% 
    df.at[cnt, "Status"] = "Done"

    DoneCnt += 1
    if DoneCnt >= 10:break

WebDriverException: Message: unknown error: DevTools returned unknown type:shared_storage_worklet
  (Session info: chrome=122.0.6261.112)
Stacktrace:
	GetHandleVerifier [0x00AB8D33+51395]
	(No symbol) [0x00A25F91]
	(No symbol) [0x008DE13A]
	(No symbol) [0x008EB131]
	(No symbol) [0x008EB4C7]
	(No symbol) [0x008DE6BC]
	(No symbol) [0x008C22ED]
	(No symbol) [0x0093F467]
	(No symbol) [0x00933E53]
	(No symbol) [0x0090C629]
	(No symbol) [0x0090D40D]
	GetHandleVerifier [0x00E364F3+3711107]
	GetHandleVerifier [0x00E758DA+3970154]
	GetHandleVerifier [0x00E70BC8+3950424]
	GetHandleVerifier [0x00B69D39+776393]
	(No symbol) [0x00A31764]
	(No symbol) [0x00A2C648]
	(No symbol) [0x00A2C7F9]
	(No symbol) [0x00A1DE20]
	BaseThreadInitThunk [0x766EFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77447C5E+286]
	RtlGetAppContainerNamedObjectPath [0x77447C2E+238]


In [2]:
# %%
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome import service
from oauth2client.service_account import ServiceAccountCredentials
from moviepy.editor import VideoFileClip
from moviepy.video.fx.all import crop
import time
import pyperclip
import random
import string
import pandas as pd
import gspread
import os
import shutil
import cv2
# %%
class GoogleSheetHandler:
    def __init__(self, credentials_file='credentials.json'):
        self.scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
        self.credentials_file = credentials_file
        self.credentials = ServiceAccountCredentials.from_json_keyfile_name(self.credentials_file, self.scope)
        self.gc = gspread.authorize(self.credentials)

    def get_sheet_data(self, spreadsheet_name, worksheet_index=0):
        spreadsheet = self.gc.open(spreadsheet_name)
        worksheet = spreadsheet.get_worksheet(worksheet_index)
        data = worksheet.get_all_values()
        df = pd.DataFrame(data[1:], columns=data[0])
        return df

    def overwrite_sheet_with_data(self, dataframe, spreadsheet_name, worksheet_index=0):
        spreadsheet = self.gc.open(spreadsheet_name)
        worksheet = spreadsheet.get_worksheet(worksheet_index)
        data = dataframe.values.tolist()
        worksheet.clear()
        # worksheet.resize(len(data), len(data[0]))
        worksheet.append_row(dataframe.columns.tolist())
        worksheet.append_rows(data)

def cropVideo(src, dest, x,y,h,w):
    clip = VideoFileClip(src)
    cropped_clip = crop(clip, x1=x, y1=y, width=w, height=h)
    cropped_clip.write_videofile(dest)
# %%
handler = GoogleSheetHandler()
df = handler.get_sheet_data('TiktokBot')
df["Status"].replace("", "Remaining", inplace=True)
handler.overwrite_sheet_with_data(df, "TiktokBot")
df

# %%
def generate_password(length=10):
    characters = string.ascii_letters + string.digits + string.punctuation
    return ''.join(random.choice(characters) for _ in range(length))

# %%
co = webdriver.ChromeOptions()

co.add_argument("--start-maximized")
co.add_argument('--disable-gpu')
co.add_argument('--disable-infobars')
co.add_argument('--no-sandbox')
# co.add_argument('--headless')
co.add_argument("--disable-popup-blocking")
co.add_argument(r"--user-data-dir=C:\Users\Dell\AppData\Roaming\Opera Software\Opera Stable")
co.add_argument(r'--profile-directory=Default') #e.g. Profile 3
co.binary_location = r"C:\Users\Dell\AppData\Local\Programs\Opera\opera.exe"
co.add_experimental_option('w3c', True)

def getWindowHandles():
    handles = driver.window_handles[1:].copy()
    del handles[1]
    return handles

# driver = webdriver.Chrome(options=co, use_subprocess=True)

webdriver_service = service.Service("operadriver.exe")
webdriver_service.start()
driver = webdriver.Remote(webdriver_service.service_url, options=co)

actions = ActionChains(driver)
driver.maximize_window()
# URLs to visit
driver.get("https://chat.openai.com/c/202a469b-f42f-4bea-a736-ae4bfc7ebed3")
time.sleep(5)
driver.implicitly_wait(10)
driver.execute_script(f'''window.open("https://www.capcut.com/my-cloud/7341722566995918853");''')
time.sleep(5)
driver.switch_to.window(getWindowHandles()[1])
driver.implicitly_wait(10)
driver.execute_script(f'''window.open("https://app.simplified.com/signup");''')
time.sleep(5)
driver.switch_to.window(getWindowHandles()[2])
driver.implicitly_wait(10)

# %%
time.sleep(0.5)
# actions.send_keys_to_element(driver.find_element(By.XPATH, "//input[@type='email']"), email).perform()
# password = generate_password()
# actions.send_keys_to_element(driver.find_element(By.XPATH, "(//input[@type='password'])"), password).perform()
actions.click(WebDriverWait(driver, 50).until(EC.presence_of_element_located((By.XPATH, "//*[contains(text(),'Sign Up with Google')]/../..")))).perform()
time.sleep(2)
try:
    driver.switch_to.window(getWindowHandles()[-1])
    actions.click(WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//*[@data-identifier='asjaisimplified@gmail.com']")))).perform()
except:
    driver.switch_to.window(getWindowHandles()[-2])
    actions.click(WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//*[@data-identifier='asjaisimplified@gmail.com']")))).perform()

time.sleep(10)

driver.switch_to.window(getWindowHandles()[2])
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@placeholder='Hint: Name of your company']"))).send_keys(random.choice(["Tiktok", "Test", "Magic", "IDK", "Youtube", "Facebook", "HomeWork", "Experiment", "Student"]))
driver.find_element(By.XPATH, "//button[@type='submit']").click()
time.sleep(1)

# %%
driver.get("https://app.simplified.com/activate/video")

driver.get("https://app.simplified.com/video/generate-ai/script-to-video")

# %%
DoneCnt = 0
for cnt, topic in  enumerate(df["Topic"]):
    if df["Status"].values.tolist()[cnt] != "Remaining":continue
    driver.switch_to.window(getWindowHandles()[0])

    prompts = [f'Write a 80 second viral educational showcase Tik Tok script about Do you know about "{topic}" Make sure the script is engaging with a good, quick hook. Keep the script relatively serious and educational. The script should ONLY include the voice, NOT the editing direction or stage direction! Start with a " and end with a ". Have paragraph breaks that stay within the". Do not do a countdown. Do not include an outro or summary. DO NOT USE EMOJIS! The script should portray the info in interactive way. Describe and make it educational. Start with "Do you know!"', "give me short caption for this too with suitable hashtags for tiktok with #DidYouKnow included"]

    driver.find_element(By.XPATH, "//textarea[@placeholder='Message ChatGPT…']").send_keys(prompts[0])
    driver.find_element(By.XPATH, "//button[@data-testid='send-button']").click()

    time.sleep(1)
    WebDriverWait(driver, 50).until(EC.presence_of_element_located((By.XPATH, "//button[@data-testid='send-button']")))
    driver.find_elements(By.XPATH, "//*[@class='flex items-center gap-1.5 rounded-md p-1 text-xs text-token-text-tertiary hover:text-token-text-primary md:invisible md:group-hover:visible md:group-[.final-completion]:visible']")[-1].click()
    script = pyperclip.paste()
    print("Script: ", script)

    driver.find_element(By.XPATH, "//textarea[@placeholder='Message ChatGPT…']").send_keys(prompts[1])
    driver.find_element(By.XPATH, "//button[@data-testid='send-button']").click()

    time.sleep(1)
    WebDriverWait(driver, 50).until(EC.presence_of_element_located((By.XPATH, "//button[@data-testid='send-button']")))
    driver.find_elements(By.XPATH, "//*[@class='flex items-center gap-1.5 rounded-md p-1 text-xs text-token-text-tertiary hover:text-token-text-primary md:invisible md:group-hover:visible md:group-[.final-completion]:visible']")[-1].click()
    title = pyperclip.paste()[1:-1]
    print("Title: ", title)

    driver.switch_to.window(getWindowHandles()[2])
    driver.get("https://app.simplified.com/video/generate-ai/script-to-video")
    driver.implicitly_wait(10)
    time.sleep(5)
    actions.click(driver.find_element(By.XPATH, "//*[@id='popover-trigger-17']")).perform()
    time.sleep(0.5)
    actions.click(WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//button[contains(text(),'AI Speakers')]")))).perform()
    time.sleep(0.5)
    actions.click(WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Matthew')]/../../../..")))).perform()
    time.sleep(1)

    actions.click(driver.find_element(By.XPATH, "//textarea[@name='title']")).perform()
    pyperclip.copy(title)
    actions.key_down(Keys.CONTROL).send_keys("v").key_up(Keys.CONTROL).perform()
    time.sleep(1)

    actions.click(driver.find_element(By.XPATH, "//textarea[@name='description']")).perform()
    pyperclip.copy(script)
    actions.key_down(Keys.CONTROL).send_keys("v").key_up(Keys.CONTROL).perform()
    time.sleep(1)

    WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Generate')]"))).click()

    # %%
    WebDriverWait(driver, 600).until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(),'Export')]/..")))
    time.sleep(5)
    WebDriverWait(driver, 600).until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(),'Export')]/.."))).click()
    time.sleep(1.2)
    WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Download')]"))).click()

    # %%
    def get_most_recent_fdmdownload():
        downloads_folder = os.path.expanduser("~/Downloads")  # Path to the Downloads folder

        while True:
            files = os.listdir(downloads_folder)
            for file in files:
                if file.endswith(".mp4.fdmdownload"):
                    return os.path.join(downloads_folder, file)
            time.sleep(1)  # Wait for 1 second

    # Function to convert .fdmdownload file to its corresponding MP4 file
    def convert_fdmdownload_to_mp4(fdmdownload_path):
        mp4_path = fdmdownload_path.replace(".fdmdownload", "")
        while not os.path.exists(mp4_path):
            time.sleep(1)  # Wait for 1 second
        # os.rename(fdmdownload_path, mp4_path)
        return mp4_path


    # %%
    # Get the path of the most recent .fdmdownload file
    recent_fdmdownload = get_most_recent_fdmdownload()
    recent_mp4 = convert_fdmdownload_to_mp4(recent_fdmdownload)
    current_directory = os.path.join(os.getcwd(), "media")
    # shutil.move(recent_mp4, os.path.join(current_directory,"temp.mp4"))

    # %%
    path = os.path.join(current_directory, title + ".mp4")
    cropVideo(recent_mp4, path, 69,244,1676,942)
    os.remove(recent_mp4)

    # %%
    # driver.switch_to.window(getWindowHandles()[1])
    # driver.find_element(By.XPATH, "//*[contains(text(),'Materials')]/../../..").click()
    # time.sleep(1)
    # driver.find_element(By.XPATH, "//input[@type='file']").send_keys(path)

    # time.sleep(10)
    # os.remove(path)

    # %% 
    df.at[cnt, "Status"] = "Done"
    handler.overwrite_sheet_with_data(df, "TiktokBot")

    DoneCnt += 1
    if DoneCnt >= 10:break

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF67D8979E2+3581954]
	(No symbol) [0x00007FF67D4A85D2]
	(No symbol) [0x00007FF67D35020B]
	(No symbol) [0x00007FF67D396F6E]
	(No symbol) [0x00007FF67D3970EC]
	(No symbol) [0x00007FF67D3D94C7]
	(No symbol) [0x00007FF67D3BA0EF]
	(No symbol) [0x00007FF67D3D6E33]
	(No symbol) [0x00007FF67D3B9E53]
	(No symbol) [0x00007FF67D38A534]
	(No symbol) [0x00007FF67D38B651]
	GetHandleVerifier [0x00007FF67D8C814D+3780461]
	GetHandleVerifier [0x00007FF67D91D5DD+4129789]
	GetHandleVerifier [0x00007FF67D915B83+4098467]
	GetHandleVerifier [0x00007FF67D5E7475+762517]
	(No symbol) [0x00007FF67D4B4FAF]
	(No symbol) [0x00007FF67D4AFD54]
	(No symbol) [0x00007FF67D4AFE90]
	(No symbol) [0x00007FF67D49E6E4]
	BaseThreadInitThunk [0x00007FF8C97C7344+20]
	RtlUserThreadStart [0x00007FF8CA2C26B1+33]


In [3]:
from moviepy.editor import VideoFileClip
from moviepy.video.fx.all import crop

def cropVideo(src, dest, x,y,h,w):
    clip = VideoFileClip(src)
    cropped_clip = crop(clip, x1=x, y1=y, width=w, height=h)
    cropped_clip.write_videofile(dest)

cropVideo(r"C:\Users\Dell\Downloads\JPH5FKVDaIMR-1710338834.mp4", r"media\Unravel the secrets of gold with me! 💰✨ #DidYouKnow #GoldFacts #PreciousMetal #HistoryLesson.mp4", 69,244,1676,942)


Moviepy - Building video media\Unravel the secrets of gold with me! 💰✨ #DidYouKnow #GoldFacts #PreciousMetal #HistoryLesson.mp4.
MoviePy - Writing audio in Unravel the secrets of gold with me! 💰✨ #DidYouKnow #GoldFacts #PreciousMetal #HistoryLessonTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video media\Unravel the secrets of gold with me! 💰✨ #DidYouKnow #GoldFacts #PreciousMetal #HistoryLesson.mp4



Moviepy - Done !
Moviepy - video ready media\Unravel the secrets of gold with me! 💰✨ #DidYouKnow #GoldFacts #PreciousMetal #HistoryLesson.mp4


In [1]:
# %%
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome import service
from oauth2client.service_account import ServiceAccountCredentials
from moviepy.editor import VideoFileClip
from moviepy.video.fx.all import crop
import time
import pyperclip
import random
import string
import pandas as pd
import gspread
import os
import shutil
import cv2
# %%
class GoogleSheetHandler:
    def __init__(self, credentials_file='credentials.json'):
        self.scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
        self.credentials_file = credentials_file
        self.credentials = ServiceAccountCredentials.from_json_keyfile_name(self.credentials_file, self.scope)
        self.gc = gspread.authorize(self.credentials)

    def get_sheet_data(self, spreadsheet_name, worksheet_index=0):
        spreadsheet = self.gc.open(spreadsheet_name)
        worksheet = spreadsheet.get_worksheet(worksheet_index)
        data = worksheet.get_all_values()
        df = pd.DataFrame(data[1:], columns=data[0])
        return df

    def overwrite_sheet_with_data(self, dataframe, spreadsheet_name, worksheet_index=0):
        spreadsheet = self.gc.open(spreadsheet_name)
        worksheet = spreadsheet.get_worksheet(worksheet_index)
        data = dataframe.values.tolist()
        worksheet.clear()
        # worksheet.resize(len(data), len(data[0]))
        worksheet.append_row(dataframe.columns.tolist())
        worksheet.append_rows(data)

def cropVideo(src, dest, x,y,h,w):
    clip = VideoFileClip(src)
    cropped_clip = crop(clip, x1=x, y1=y, width=w, height=h)
    cropped_clip.write_videofile(dest)
# %%
handler = GoogleSheetHandler()
df = handler.get_sheet_data('TiktokBot')
df["Status"].replace("", "Remaining", inplace=True)
handler.overwrite_sheet_with_data(df, "TiktokBot")
df

# %%
def generate_password(length=10):
    characters = string.ascii_letters + string.digits + string.punctuation
    return ''.join(random.choice(characters) for _ in range(length))

# %%
co = webdriver.ChromeOptions()

co.add_argument("--start-maximized")
co.add_argument('--disable-gpu')
co.add_argument('--disable-infobars')
co.add_argument('--no-sandbox')
# co.add_argument('--headless')
co.add_argument("--disable-popup-blocking")
co.add_argument(r"--user-data-dir=C:\Users\Dell\AppData\Roaming\Opera Software\Opera Stable")
co.add_argument(r'--profile-directory=Default') #e.g. Profile 3
co.binary_location = r"C:\Users\Dell\AppData\Local\Programs\Opera\opera.exe"
co.add_experimental_option('w3c', True)

def getWindowHandles():
    handles = driver.window_handles[1:].copy()
    del handles[1]
    return handles

# driver = webdriver.Chrome(options=co, use_subprocess=True)

webdriver_service = service.Service("operadriver.exe")
webdriver_service.start()
driver = webdriver.Remote(webdriver_service.service_url, options=co)

actions = ActionChains(driver)
driver.maximize_window()
# URLs to visit
driver.get("https://chat.openai.com/c/202a469b-f42f-4bea-a736-ae4bfc7ebed3")
time.sleep(5)
driver.implicitly_wait(10)
driver.execute_script(f'''window.open("https://www.capcut.com/my-cloud/7341722566995918853");''')
time.sleep(5)
driver.switch_to.window(getWindowHandles()[1])
driver.implicitly_wait(10)
driver.execute_script(f'''window.open("https://app.simplified.com/signup");''')
time.sleep(5)
driver.switch_to.window(getWindowHandles()[2])
driver.implicitly_wait(10)

# %%
time.sleep(0.5)
# actions.send_keys_to_element(driver.find_element(By.XPATH, "//input[@type='email']"), email).perform()
# password = generate_password()
# actions.send_keys_to_element(driver.find_element(By.XPATH, "(//input[@type='password'])"), password).perform()
actions.click(WebDriverWait(driver, 50).until(EC.presence_of_element_located((By.XPATH, "//*[contains(text(),'Sign Up with Google')]/../..")))).perform()
time.sleep(2)
try:
    driver.switch_to.window(getWindowHandles()[-1])
    actions.click(WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//*[@data-identifier='asjaisimplified@gmail.com']")))).perform()
except:
    driver.switch_to.window(getWindowHandles()[-2])
    actions.click(WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//*[@data-identifier='asjaisimplified@gmail.com']")))).perform()

time.sleep(10)

driver.switch_to.window(getWindowHandles()[2])
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@placeholder='Hint: Name of your company']"))).send_keys(random.choice(["Tiktok", "Test", "Magic", "IDK", "Youtube", "Facebook", "HomeWork", "Experiment", "Student"]))
driver.find_element(By.XPATH, "//button[@type='submit']").click()
time.sleep(1)

# %%
driver.get("https://app.simplified.com/activate/video")

driver.get("https://app.simplified.com/video/generate-ai/script-to-video")

# %%
DoneCnt = 0
for cnt, topic in  enumerate(df["Topic"]):
    if df["Status"].values.tolist()[cnt] != "Remaining":continue
    driver.switch_to.window(getWindowHandles()[0])

    prompts = [f'Write a 80 second viral educational showcase Tik Tok script about Do you know about "{topic}" Make sure the script is engaging with a good, quick hook. Keep the script relatively serious and educational. The script should ONLY include the voice, NOT the editing direction or stage direction! Start with a " and end with a ". Have paragraph breaks that stay within the". Do not do a countdown. Do not include an outro or summary. DO NOT USE EMOJIS! The script should portray the info in interactive way. Describe and make it educational. Start with "Do you know!"', "give me short caption for this too with suitable hashtags for tiktok with #DidYouKnow included"]

    driver.find_element(By.XPATH, "//textarea[@placeholder='Message ChatGPT…']").send_keys(prompts[0])
    driver.find_element(By.XPATH, "//button[@data-testid='send-button']").click()

    time.sleep(1)
    WebDriverWait(driver, 50).until(EC.presence_of_element_located((By.XPATH, "//button[@data-testid='send-button']")))
    driver.find_elements(By.XPATH, "//*[@class='flex items-center gap-1.5 rounded-md p-1 text-xs text-token-text-tertiary hover:text-token-text-primary md:invisible md:group-hover:visible md:group-[.final-completion]:visible']")[-1].click()
    script = pyperclip.paste()
    print("Script: ", script)

    driver.find_element(By.XPATH, "//textarea[@placeholder='Message ChatGPT…']").send_keys(prompts[1])
    driver.find_element(By.XPATH, "//button[@data-testid='send-button']").click()

    time.sleep(1)
    WebDriverWait(driver, 50).until(EC.presence_of_element_located((By.XPATH, "//button[@data-testid='send-button']")))
    driver.find_elements(By.XPATH, "//*[@class='flex items-center gap-1.5 rounded-md p-1 text-xs text-token-text-tertiary hover:text-token-text-primary md:invisible md:group-hover:visible md:group-[.final-completion]:visible']")[-1].click()
    title = pyperclip.paste()[1:-1]
    print("Title: ", title)

    driver.switch_to.window(getWindowHandles()[2])
    driver.get("https://app.simplified.com/video/generate-ai/script-to-video")
    driver.implicitly_wait(10)
    time.sleep(5)
    actions.click(driver.find_element(By.XPATH, "//*[@id='popover-trigger-17']")).perform()
    time.sleep(0.5)
    actions.click(WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//button[contains(text(),'AI Speakers')]")))).perform()
    time.sleep(0.5)
    actions.click(WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Matthew')]/../../../..")))).perform()
    time.sleep(1)

    actions.click(driver.find_element(By.XPATH, "//textarea[@name='title']")).perform()
    pyperclip.copy(title)
    actions.key_down(Keys.CONTROL).send_keys("v").key_up(Keys.CONTROL).perform()
    time.sleep(1)

    actions.click(driver.find_element(By.XPATH, "//textarea[@name='description']")).perform()
    pyperclip.copy(script)
    actions.key_down(Keys.CONTROL).send_keys("v").key_up(Keys.CONTROL).perform()
    time.sleep(1)

    WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Generate')]"))).click()

    # %%
    WebDriverWait(driver, 600).until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(),'Export')]/..")))
    time.sleep(5)
    WebDriverWait(driver, 600).until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(),'Export')]/.."))).click()
    time.sleep(1.2)
    WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Download')]"))).click()

    # %%
    def get_most_recent_fdmdownload():
        downloads_folder = os.path.expanduser("~/Downloads")  # Path to the Downloads folder

        while True:
            files = os.listdir(downloads_folder)
            for file in files:
                if file.endswith(".mp4.fdmdownload"):
                    return os.path.join(downloads_folder, file)
            time.sleep(1)  # Wait for 1 second

    # Function to convert .fdmdownload file to its corresponding MP4 file
    def convert_fdmdownload_to_mp4(fdmdownload_path):
        mp4_path = fdmdownload_path.replace(".fdmdownload", "")
        while not os.path.exists(mp4_path):
            time.sleep(1)  # Wait for 1 second
        # os.rename(fdmdownload_path, mp4_path)
        return mp4_path


    # %%
    # Get the path of the most recent .fdmdownload file
    recent_fdmdownload = get_most_recent_fdmdownload()
    recent_mp4 = convert_fdmdownload_to_mp4(recent_fdmdownload)
    current_directory = os.path.join(os.getcwd(), "media")
    # shutil.move(recent_mp4, os.path.join(current_directory,"temp.mp4"))

    # %%
    path = os.path.join(current_directory, title + ".mp4")
    cropVideo(recent_mp4, path, 69,244,1676,942)
    os.remove(recent_mp4)

    # %%
    # driver.switch_to.window(getWindowHandles()[1])
    # driver.find_element(By.XPATH, "//*[contains(text(),'Materials')]/../../..").click()
    # time.sleep(1)
    # driver.find_element(By.XPATH, "//input[@type='file']").send_keys(path)

    # time.sleep(10)
    # os.remove(path)

    # %% 
    df.at[cnt, "Status"] = "Done"
    handler.overwrite_sheet_with_data(df, "TiktokBot")

    DoneCnt += 1
    if DoneCnt >= 10:break

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF788D879E2+3581954]
	(No symbol) [0x00007FF7889985D2]
	(No symbol) [0x00007FF78884020B]
	(No symbol) [0x00007FF788886F6E]
	(No symbol) [0x00007FF7888870EC]
	(No symbol) [0x00007FF7888C94C7]
	(No symbol) [0x00007FF7888AA0EF]
	(No symbol) [0x00007FF7888C6E33]
	(No symbol) [0x00007FF7888A9E53]
	(No symbol) [0x00007FF78887A534]
	(No symbol) [0x00007FF78887B651]
	GetHandleVerifier [0x00007FF788DB814D+3780461]
	GetHandleVerifier [0x00007FF788E0D5DD+4129789]
	GetHandleVerifier [0x00007FF788E05B83+4098467]
	GetHandleVerifier [0x00007FF788AD7475+762517]
	(No symbol) [0x00007FF7889A4FAF]
	(No symbol) [0x00007FF78899FD54]
	(No symbol) [0x00007FF78899FE90]
	(No symbol) [0x00007FF78898E6E4]
	BaseThreadInitThunk [0x00007FF8E0C07344+20]
	RtlUserThreadStart [0x00007FF8E19226B1+33]


In [32]:
path

'd:\\Python Scripts\\Ongoing\\VideoCreationTiktok\\media\\Exploring the wonders of apples! 🍎 #DidYouKnow #AppleFacts.mp4'

In [60]:
# driver.switch_to.window(getWindowHandles()[-1])
for d in driver.find_elements(By.XPATH, "//input"):
    try:d.send_keys(path)
    except:continue

In [22]:
s = "MCMXCIV"

total = 0
RomanMap = {
    'M':1000,
    'D':500,
    'C':100,
    'L':50,
    'X':10,
    'V':5,
    'I':1
}
seq = list(RomanMap.keys())[::-1]

for cnt, i in enumerate(s):
    if seq.index(i) > seq.index(s[cnt-1]) and cnt >= 1:
        total -= RomanMap[i]
        print("Kat Giya", RomanMap[i], seq.index(i), seq.index(s[cnt-1]))
    else:
        total += RomanMap[i]

total

Kat Giya 1000 6 4
Kat Giya 100 4 2
Kat Giya 5 1 0


6

In [49]:
for w in getWindowHandles():
    driver.switch_to.window(w)
    print(w, driver.current_url)

F7E1A355ADA4C836AE7195302537206A https://chat.openai.com/c/202a469b-f42f-4bea-a736-ae4bfc7ebed3
0655854ACFD2EC500AFFCB5F1DFB79AF https://app.simplified.com/videos/9139511/92137a93-2395-465f-98da-809b08a87833/exploring-the-wonders-of-apples-didyouknow-applefacts
4E9C22851A38741AA4D86C1499340668 chrome://startpageshared/
5633B9766AFB037693B50B88D80829D3 https://www.google.com/search?q=upload+files+to+capcut+using+python&client=opera&hs=eiS&sca_esv=992a11cd8a65ea7e&sxsrf=ACQVn09lUnanCW-gLnY0ioRp06RdAya86w%3A1710017329012&ei=McvsZZ4X7q_k2g_dyL_IBw&ved=0ahUKEwje89H7huiEAxXuF1kFHV3kD3kQ4dUDCBA&uact=5&oq=upload+files+to+capcut+using+python&gs_lp=Egxnd3Mtd2l6LXNlcnAiI3VwbG9hZCBmaWxlcyB0byBjYXBjdXQgdXNpbmcgcHl0aG9uMgQQABhHMgQQABhHMgQQABhHMgQQABhHMgQQABhHMgQQABhHMgQQABhHMgQQABhHSO1FUMIJWP9DcAB4ApABAJgBAKABAKoBALgBA8gBAPgBAZgCAaACB6gCAJgDAIgGAZAGCJIHATGgBwA&sclient=gws-wiz-serp#ip=1
7DA7D7F90391E09F5D8C73CB3638285E https://www.google.com/search?client=opera&q=tiktpk+shedular&sourceid=opera&ie=UTF